In [28]:
import pandas as pd
from sklearn.metrics import *
from imblearn.metrics import specificity_score

In [21]:
train_data = pd.read_csv('C:\\Users\\gamma\\Desktop\\Bootcamp\\13-Tree_decision\\tree-decision\\src\\datos_entrenados.csv')
test_data = pd.read_csv('C:\\Users\\gamma\\Desktop\\Bootcamp\\13-Tree_decision\\tree-decision\\src\\datos_testeo.csv')

In [22]:
X_train = train_data.drop(["Outcome"], axis = 1)
y_train = train_data["Outcome"]
X_test = test_data.drop(["Outcome"], axis = 1)
y_test = test_data["Outcome"]

In [23]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators = 500, learning_rate = 0.001, random_state = 42)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.001, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [37]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [33]:
from sklearn.model_selection import GridSearchCV

# Primero, creamos nuestro detective especial, el XGBClassifier.
modelo_xgb = XGBClassifier(random_state=42)

# Ahora, vamos a preparar un conjunto de llaves mágicas (hiperparámetros) para probar en la cerradura del misterio.
hyperparams = {
    "learning_rate": [0.01, 0.1, 0.2],  # Esto es como la velocidad a la que el detective aprende nuevas pistas.
    "max_depth": [3, 5, 7],             # Esto es qué tan profundo el detective investiga en cada pista.
    "min_child_weight": [1, 3, 5],      # Esto es cuántas pistas mínimas necesita el detective para tomar una decisión.
    "subsample": [0.5, 0.7, 1.0],       # Esto es qué parte de las pistas el detective considera en cada paso.
    "colsample_bytree": [0.5, 0.7, 1.0],# Esto es qué parte de las pistas (columnas) el detective usa para construir cada árbol de pistas.
    "n_estimators": [100, 200, 300]     # Esto es cuántos árboles de pistas el detective va a construir para resolver el misterio.
}

# GridSearchCV es como un robot que prueba todas las combinaciones de llaves por nosotros.
grid = GridSearchCV(modelo_xgb, hyperparams, scoring='accuracy', cv=5)

# Le decimos al robot que empiece a trabajar y pruebe todas las combinaciones.
grid.fit(X_train, y_train)

# El robot nos dirá cuál es la mejor combinación de llaves para resolver el misterio.
mejores_parametros = grid.best_params_
print(f"La mejor combinación de pistas es: {mejores_parametros}")
print(f"Y con estas pistas, la precisión del detective es del {grid.best_score_ * 100:.2f}%")


La mejor combinación de pistas es: {'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 300, 'subsample': 0.5}
Y con estas pistas, la precisión del detective es del 78.34%


In [52]:
modelo_optimizado = XGBClassifier(**mejores_parametros)

modelo_optimizado.fit(X_train, y_train)

entrenamiento_mejorado = modelo_optimizado.predict(X_train)

testeo_mejorado = modelo_optimizado.predict(X_test)

In [54]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, testeo_mejorado)

0.7532467532467533

In [55]:
def get_metrics(y_train, y_test, y_pred_train, y_pred_test):
    # Calcular métricas para el conjunto de entrenamiento
    train_accuracy = accuracy_score(y_train, y_pred_train)
    train_f1 = f1_score(y_train, y_pred_train)
    train_auc = roc_auc_score(y_train, y_pred_train)
    train_precision = precision_score(y_train, y_pred_train)
    train_recall = recall_score(y_train, y_pred_train)
    train_specificity = specificity_score(y_train, y_pred_train)
    # Calcular métricas para el conjunto de prueba
    test_accuracy = accuracy_score(y_test, y_pred_test)
    test_f1 = f1_score(y_test, y_pred_test)
    test_auc = roc_auc_score(y_test, y_pred_test)
    test_precision = precision_score(y_test, y_pred_test)
    test_recall = recall_score(y_test, y_pred_test)
    test_specificity = specificity_score(y_test, y_pred_test)
    # Calcular la diferencia entre métricas de entrenamiento y prueba
    diff_accuracy = train_accuracy - test_accuracy
    diff_f1 = train_f1 - test_f1
    diff_auc = train_auc - test_auc
    diff_precision = train_precision - test_precision
    diff_recall = train_recall - test_recall
    diff_specificity = train_specificity - test_specificity
    # Crear un DataFrame con los resultados
    metrics_df = pd.DataFrame([[train_accuracy, train_f1, train_auc, train_precision, train_recall, train_specificity],[test_accuracy, test_f1, test_auc, test_precision, test_recall, test_specificity],[diff_accuracy, diff_f1, diff_auc, diff_precision, diff_recall, diff_specificity]],
                              columns = ['Accuracy', 'F1', 'AUC', 'Precision', 'Recall', 'Specificity'],
                              index = ['Train','Test', 'Diferencia'])
    return metrics_df

In [59]:
get_metrics(y_train, y_test, entrenamiento_mejorado, testeo_mejorado)

,Accuracy,F1,AUC,Precision,Recall,Specificity
Train,0.895765,0.834197,0.862972,0.930636,0.755869,0.970075
Test,0.753247,0.648148,0.727273,0.660377,0.636364,0.818182
Diferencia,0.142519,0.186049,0.135699,0.270258,0.119505,0.151893
